# Start Up

Run the cells below. They will open a browser window through selenium. (check your open windows if you don't notice it. It will be a firefox page with an orange url bar.)

Log into your ESPN account and navigate to the draft.

Make sure to keep everything in a single tab. Do not open links as new windows, do not open links as new tabs. This should be the default behavior if you just click on things normally (but no promises - this is hacked together at best...)

### Updating the probabilities

The default score/pick probability info is for 10 team, standard (non-ppr) scoring.

You can update the probabilities with different parameters ```yahoowizard.create_prob_pickle(teams, outputname, mapping_path, scoring, outputpath)``` where teams is the number of teams in your league, and scoring is one of these four entries:
 - 'standard'
 - 'ppr'
 - 'half-ppr'
 - '2qb

The other parameters deal with file mappings.
 
Follow this link to see the source data [Fantasy Football Calculator](https://fantasyfootballcalculator.com/scenario-calculator)

### Updating/Changing the Projections

You can call ```yahoowizard.process_projections(projection_path, idmap_path, output_path)```
to update or change the projections. ```projection_path``` is the direction to the projection files. I use [fantasyfootballanalytics.net](https://www.fantasyfootballanalytics.net)

You can use any source of projections as long as you can map them to the ESPN IDs. 

Required columns:
 - 'position'
 - 'player'
 - 'points'
 - 'adp'
 
This needs more development, specifically:
 - mapping tables to cross-reference all the different sources to some sort of master ID
 - It would also be useful to map the player IDs to the different providers.
 - making the column naming more flexible 
 
 
 # Current Use.
 tells you how to use it... tbd

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd 
import numpy as np
import yahoowizard as yw
import matplotlib.pyplot as plt

# projections

In [ ]:
proj_name = 'yahoo_players.pkl'
prob_name = 'pick_prob_12_halfppr.pkl'

In [ ]:
yw.process_projections('yahoo_projections.csv',
                       'espn_ffa_map.csv',
                       proj_name)

In [ ]:
yw.create_prob_pickle(12,
                       'half-ppr',
                       prob_name,
                       'probmap.csv')

In [ ]:
projections = pd.read_pickle(proj_name)
probabilities = pd.read_pickle(prob_name)

In [ ]:
league_id = 10302553
your_first_pick = 4

In [ ]:
draft = yw.YahooDraft(league_id, your_first_pick)

In [ ]:
proj1 = draft.update(projections)

In [ ]:
oc = yw.tell_me_what_to_do(projections,probabilities,draft)
cols = ['player','team','position','points','oc_raw_0','oc_adj_0','pb_0','oc_raw_1','oc_adj_1','pb_1']
print('\nRoster Needs')
print(draft.pos_needs.loc[['QB','RB','WR','TE','DST','K']])
print('\n')
oc[cols].sort_values('oc_adj_1',ascending=False).round(2).head(10)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
oc.groupby('position').apply(lambda x: x.sort_values('oc_adj_1',ascending=False)['oc_adj_1'].reset_index(drop=True).head(5).rename(x['pos'].unique()[0]).plot(ax=ax, legend=True))
plt.axhline(0, color='k')
plt.show()

In [ ]:
projections = yw.blacklist_player('Leonard Fournette','JAC',projections)
# projections = yw.unblacklist_player('Leonard Fournette','JAC',projections)